In [1]:
import duckdb
import pandas as pd

# Connect to the database
connection = duckdb.connect("../data/transport_data.db", read_only=False)


query = """ CREATE TABLE IF NOT EXISTS services AS SELECT * FROM '../data/delay_data/*.csv'"""
# @TODO: Add logic to rename the columns
connection.execute(query)

r = connection.sql("DESCRIBE services")

In [2]:
import duckdb

connection = duckdb.connect("../transport_data.db", read_only=False)

In [5]:
# Get the count of all entries
query = """
SELECT *
FROM services
LIMIT 10
"""
result = connection.sql(query).df()
result

,OPERATING_DAY,TRIP_IDENTIFIER,OPERATOR_ID,OPERATOR_ABK,OPERATOR_NAME,PRODUCT_ID,LINE_ID,LINE_TEXT,CYCLE_ID,TRANSPORT_MODE_TEXT,...,CANCELLED_TF,BPUIC,STOP_NAME,ARRIVAL_TIME,ARRIVAL_PREDICTION,ARRIVAL_PREDICTION_STATUS,DEPARTURE_TIME,DEPARTURE_PREDICTION,DEPARTURE_PREDICTION_STATUS,THROUGH_TF
0,2024-05-01,70681-8006C5-8000774-165900,80:06____,DB,DB Regio AG Baden-Württemberg,Zug,70681,RE7,None,RE,...,True,8500090,Basel Bad Bf,01.05.2024 19:23,NaT,UNBEKANNT,None,NaT,UNBEKANNT,False
1,2024-05-01,80:800631:17238:000,80:800631,DB,DB Regio AG Baden-Württemberg,Zug,17238,RB,None,RB,...,False,8500090,Basel Bad Bf,01.05.2024 07:05,2024-05-01 07:06:00,PROGNOSE,None,NaT,None,False
2,2024-05-01,80:800631:17239:000,80:800631,DB,DB Regio AG Baden-Württemberg,Zug,17239,RB,None,RB,...,False,8500090,Basel Bad Bf,None,NaT,None,01.05.2024 07:53,2024-05-01 07:53:00,PROGNOSE,False
3,2024-05-01,80:800631:17241:000,80:800631,DB,DB Regio AG Baden-Württemberg,Zug,17241,RB,None,RB,...,False,8500090,Basel Bad Bf,None,NaT,None,01.05.2024 08:53,2024-05-01 08:53:00,PROGNOSE,False
4,2024-05-01,80:800631:17243:000,80:800631,DB,DB Regio AG Baden-Württemberg,Zug,17243,RB,None,RB,...,True,8500090,Basel Bad Bf,None,NaT,UNBEKANNT,01.05.2024 09:53,NaT,UNBEKANNT,False
5,2024-05-01,80:800631:17244:000,80:800631,DB,DB Regio AG Baden-Württemberg,Zug,17244,RB,None,RB,...,False,8500090,Basel Bad Bf,01.05.2024 08:08,2024-05-01 08:08:00,PROGNOSE,None,NaT,None,False
6,2024-05-01,80:800631:17245:000,80:800631,DB,DB Regio AG Baden-Württemberg,Zug,17245,RB,None,RB,...,False,8500090,Basel Bad Bf,None,NaT,None,01.05.2024 10:53,2024-05-01 10:53:00,PROGNOSE,False
7,2024-05-01,80:800631:17248:000,80:800631,DB,DB Regio AG Baden-Württemberg,Zug,17248,RB,None,RB,...,True,8500090,Basel Bad Bf,01.05.2024 09:08,NaT,UNBEKANNT,None,NaT,UNBEKANNT,False
8,2024-05-01,80:800631:17249:000,80:800631,DB,DB Regio AG Baden-Württemberg,Zug,17249,RB,None,RB,...,False,8500090,Basel Bad Bf,None,NaT,None,01.05.2024 11:53,2024-05-01 11:54:00,PROGNOSE,False
9,2024-05-01,80:800631:17250:000,80:800631,DB,DB Regio AG Baden-Württemberg,Zug,17250,RB,None,RB,...,False,8500090,Basel Bad Bf,01.05.2024 10:08,2024-05-01 10:09:00,PROGNOSE,None,NaT,None,False


In [2]:
# Rename the columns -> for later use (use when creating the table in the DB)
df = pd.DataFrame()
df.rename(
    columns={
        "BETRIEBSTAG": "OPERATING_DAY",
        "FAHRT_BEZEICHNER": "TRIP_IDENTIFIER",
        "BETREIBER_ID": "OPERATOR_ID",
        "BETREIBER_ABK": "OPERATOR_ABK",
        "BETREIBER_NAME": "OPERATOR_NAME",
        "PRODUKT_ID": "PRODUCT_ID",
        "LINIEN_ID": "LINE_ID",
        "LINIEN_TEXT": "LINE_TEXT",
        "UMLAUF_ID": "CYCLE_ID",
        "VERKEHRSMITTEL_TEXT": "TRANSPORT_MODE_TEXT",
        "ZUSATZFAHRT_TF": "ADDITIONAL_TRIP_TF",
        "FAELLT_AUS_TF": "CANCELLED_TF",
        "BPUIC": "BPUIC",
        "HALTESTELLEN_NAME": "STOP_NAME",
        "ANKUNFTSZEIT": "ARRIVAL_TIME",
        "AN_PROGNOSE": "ARRIVAL_PREDICTION",
        "AN_PROGNOSE_STATUS": "ARRIVAL_PREDICTION_STATUS",
        "ABFAHRTSZEIT": "DEPARTURE_TIME",
        "AB_PROGNOSE": "DEPARTURE_PREDICTION",
        "AB_PROGNOSE_STATUS": "DEPARTURE_PREDICTION_STATUS",
        "DURCHFAHRT_TF": "THROUGH_TF",
    },
    inplace=True,
)

In [3]:
# Get the count of all entries
query = """
SELECT count(*)
FROM services
WHERE PRODUKT_ID='Zug'
"""
result = connection.sql(query).df()

# Print the result
print("count:")
print(result)

count:
   count_star()
0       4772714


In [4]:
# Get the 20 most delayed trains
query = """
SELECT LINIEN_ID, AVG(epoch((AN_PROGNOSE - try_strptime(ANKUNFTSZEIT, '%d.%m.%Y %H:%M'))))/3600 as avg_delay, count(*) as count
FROM services
WHERE PRODUKT_ID='Zug'
AND BETREIBER_NAME='Schweizerische Bundesbahnen SBB'
AND AN_PROGNOSE_STATUS='REAL'
GROUP BY LINIEN_ID
ORDER BY 2 DESC
LIMIT 40
"""
result = connection.sql(query).df()

pd.set_option("display.max_columns", None)
print("20 most delayed trains:")
result

20 most delayed trains:


,LINIEN_ID,avg_delay,count
0,17790,1.846713,6
1,17786,1.442500,4
2,466,1.420501,66
3,13491,1.254611,5
4,17788,1.085722,5
5,31216,1.030556,1
6,17784,0.968056,2
7,31438,0.906815,15
8,70196,0.848472,2
9,73290,0.830625,8


In [5]:
# get the Betreiber with name 'Schweizerische Bundesbahnen SBB'
query = """
SELECT *
FROM services
WHERE PRODUKT_ID='Zug'
AND BETREIBER_NAME='Schweizerische Bundesbahnen SBB'
"""
result = connection.sql(query).df()
result.rename(
    columns={
        "BETRIEBSTAG": "OPERATING_DAY",
        "FAHRT_BEZEICHNER": "TRIP_IDENTIFIER",
        "BETREIBER_ID": "OPERATOR_ID",
        "BETREIBER_ABK": "OPERATOR_ABK",
        "BETREIBER_NAME": "OPERATOR_NAME",
        "PRODUKT_ID": "PRODUCT_ID",
        "LINIEN_ID": "LINE_ID",
        "LINIEN_TEXT": "LINE_TEXT",
        "UMLAUF_ID": "CYCLE_ID",
        "VERKEHRSMITTEL_TEXT": "TRANSPORT_MODE_TEXT",
        "ZUSATZFAHRT_TF": "ADDITIONAL_TRIP_TF",
        "FAELLT_AUS_TF": "CANCELLED_TF",
        "BPUIC": "BPUIC",
        "HALTESTELLEN_NAME": "STOP_NAME",
        "ANKUNFTSZEIT": "ARRIVAL_TIME",
        "AN_PROGNOSE": "ARRIVAL_PREDICTION",
        "AN_PROGNOSE_STATUS": "ARRIVAL_PREDICTION_STATUS",
        "ABFAHRTSZEIT": "DEPARTURE_TIME",
        "AB_PROGNOSE": "DEPARTURE_PREDICTION",
        "AB_PROGNOSE_STATUS": "DEPARTURE_PREDICTION_STATUS",
        "DURCHFAHRT_TF": "THROUGH_TF",
    },
    inplace=True,
)
result

,OPERATING_DAY,TRIP_IDENTIFIER,OPERATOR_ID,OPERATOR_ABK,OPERATOR_NAME,PRODUCT_ID,LINE_ID,LINE_TEXT,CYCLE_ID,TRANSPORT_MODE_TEXT,ADDITIONAL_TRIP_TF,CANCELLED_TF,BPUIC,STOP_NAME,ARRIVAL_TIME,ARRIVAL_PREDICTION,ARRIVAL_PREDICTION_STATUS,DEPARTURE_TIME,DEPARTURE_PREDICTION,DEPARTURE_PREDICTION_STATUS,THROUGH_TF
0,2024-09-01,ch:1:sjyid:100001:1007-001,85:11,SBB,Schweizerische Bundesbahnen SBB,Zug,1007,IC,None,IC,False,False,8503424,Schaffhausen,None,NaT,None,01.09.2024 05:47,2024-09-01 05:47:34,REAL,False
1,2024-09-01,ch:1:sjyid:100001:1007-001,85:11,SBB,Schweizerische Bundesbahnen SBB,Zug,1007,IC,None,IC,False,False,8503000,Zürich HB,01.09.2024 06:26,2024-09-01 06:26:05,REAL,None,NaT,None,False
2,2024-09-01,ch:1:sjyid:100001:1009-001,85:11,SBB,Schweizerische Bundesbahnen SBB,Zug,1009,IC,None,IC,False,False,8503424,Schaffhausen,None,NaT,None,01.09.2024 06:47,2024-09-01 06:47:18,REAL,False
3,2024-09-01,ch:1:sjyid:100001:1009-001,85:11,SBB,Schweizerische Bundesbahnen SBB,Zug,1009,IC,None,IC,False,False,8503000,Zürich HB,01.09.2024 07:26,2024-09-01 07:26:36,REAL,None,NaT,None,False
4,2024-09-01,ch:1:sjyid:100001:1011-001,85:11,SBB,Schweizerische Bundesbahnen SBB,Zug,1011,IC,None,IC,False,False,8503424,Schaffhausen,None,NaT,None,01.09.2024 07:47,2024-09-01 07:47:42,REAL,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1895692,2024-09-30,ch:1:sjyid:100001:992-001,85:11,SBB,Schweizerische Bundesbahnen SBB,Zug,992,IC61,None,IC,False,False,8507492,Interlaken Ost,None,NaT,None,30.09.2024 23:00,2024-09-30 23:00:50,REAL,False
1895693,2024-09-30,ch:1:sjyid:100001:992-001,85:11,SBB,Schweizerische Bundesbahnen SBB,Zug,992,IC61,None,IC,False,False,8507493,Interlaken West,30.09.2024 23:04,2024-09-30 23:04:20,REAL,30.09.2024 23:05,2024-09-30 23:06:01,REAL,False
1895694,2024-09-30,ch:1:sjyid:100001:992-001,85:11,SBB,Schweizerische Bundesbahnen SBB,Zug,992,IC61,None,IC,False,False,8507483,Spiez,30.09.2024 23:22,2024-09-30 23:21:32,REAL,30.09.2024 23:22,2024-09-30 23:22:54,REAL,False
1895695,2024-09-30,ch:1:sjyid:100001:992-001,85:11,SBB,Schweizerische Bundesbahnen SBB,Zug,992,IC61,None,IC,False,False,8507100,Thun,30.09.2024 23:32,2024-09-30 23:31:30,REAL,30.09.2024 23:33,2024-09-30 23:33:53,REAL,False


In [18]:
query = """
SELECT LINIEN_ID, HALTESTELLEN_NAME, BPUIC
FROM services
WHERE LINIEN_ID IN ['17790']
--AND AN_PROGNOSE_STATUS='REAL'
LIMIT 40
"""
result = connection.sql(query).df()

pd.set_option("display.max_columns", None)
result

,LINIEN_ID,HALTESTELLEN_NAME,BPUIC
0,17790,St. Margrethen SG,8506314
1,17790,Winterthur,8506000
2,17790,Zürich HB,8503000
3,17790,St. Margrethen SG,8506314
4,17790,Winterthur,8506000
5,17790,Zürich HB,8503000


In [24]:
stops = [8503000]
query = f"""
SELECT BPUIC, HALTESTELLEN_NAME, LINIEN_ID
FROM services
WHERE BPUIC IN {stops}
AND HALTESTELLEN_NAME NOT NULL
AND PRODUKT_ID='Zug'
LIMIT 40
"""
result = connection.sql(query).df()

pd.set_option("display.max_columns", None)
result

,BPUIC,HALTESTELLEN_NAME,LINIEN_ID
0,8503000,Zürich HB,1007
1,8503000,Zürich HB,1009
2,8503000,Zürich HB,1011
3,8503000,Zürich HB,10318
4,8503000,Zürich HB,10320
5,8503000,Zürich HB,10322
6,8503000,Zürich HB,10328
7,8503000,Zürich HB,10334
8,8503000,Zürich HB,10340
9,8503000,Zürich HB,10368


In [29]:
query = """
SELECT BPUIC, FIRST(HALTESTELLEN_NAME), LINIEN_ID
FROM services
WHERE LINIEN_ID='1007'
AND HALTESTELLEN_NAME NOT NULL
AND PRODUKT_ID='Zug'
GROUP BY 3, 1
LIMIT 40
"""
result = connection.sql(query).df()

result

,BPUIC,"""first""(HALTESTELLEN_NAME)",LINIEN_ID
0,8503400,Bülach,1007
1,8503404,Rafz,1007
2,8503000,Zürich HB,1007
3,8503421,Jestetten,1007
4,8503424,Schaffhausen,1007


In [14]:
stop_name = "Zürich"
query = f"""SELECT STOP_NAME, count() FROM services WHERE STOP_NAME  LIKE '%{stop_name}%' and STOP_NAME NOT LIKE '%,%' group by 1"""
df = connection.execute(query).df()

In [15]:
df

,STOP_NAME,count_star()
0,Zürich HB,349924
1,Zürich Giesshübel,25219
2,Zürich Binz,25999
3,Zürich Tiefenbrunnen,28576
4,Zürich Wollishofen,28279
5,Zürich Wiedikon,40715
6,Zürich Stadelhofen,125545
7,Zürich Wipkingen,13234
8,Zürich Leimbach,25240
9,Zürich Manegg,25219


In [ ]:
# Close the connection
connection.close()